# SEC 10-K RAG System - Colab Demo

**Author:** Indhra  
**Assignment:** LLM + RAG Hands-On Coding Test

This notebook demonstrates a RAG system for answering questions from Apple 2024 and Tesla 2023 10-K SEC filings.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/indhra/llm_rag_sec_findings/blob/main/notebooks/RAG_SEC_10K.ipynb)

## 1. Setup

Clone the repository and install dependencies.

In [3]:
# Clone the repository
!git clone https://github.com/indhra/llm_rag_sec_findings.git
%cd llm_rag_sec_findings

Cloning into 'llm_rag_sec_findings'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 157 (delta 72), reused 138 (delta 56), pack-reused 0 (from 0)
Receiving objects: 100% (157/157), 1.72 MiB | 5.89 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/Users/indhra/Machine_learning/Resumes_Indhra/ABB_JAN26/notebooks/llm_rag_sec_findings


In [4]:
# Install dependencies
!pip install -q pymupdf tiktoken sentence-transformers faiss-cpu rank-bm25 groq python-dotenv tqdm

zsh:1: command not found: pip


In [5]:
"https://console.groq.com/keys"

'https://console.groq.com/keys'

In [ ]:
# Set your Groq API key (free at console.groq.com)
import os
from getpass import getpass

# Option 1: Enter manually (strips whitespace to prevent errors)
api_key = getpass("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = api_key.strip()

# Option 2: Use Colab secrets (recommended)
# from google.colab import userdata
# os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY').strip()

In [8]:
# Validate Groq API key
print("🔑 Validating Groq API key...")
try:
    from groq import Groq

    # Ensure the API key is stripped of any leading/trailing whitespace or newlines
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if groq_api_key:
        client = Groq(api_key=groq_api_key.strip())
    else:
        raise ValueError("GROQ_API_KEY environment variable is not set.")

    # Test with a simple request
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "Hi"}],
        max_tokens=5
    )

    print("✅ API key is valid and working!")
    print(f"   Model: {response.model}")
    print(f"   Response: {response.choices[0].message.content}")

except Exception as e:
    print(f"❌ API key validation failed: {str(e)}")
    print("\nPlease check your API key and try again.")
    print("Get a free key at: https://console.groq.com/keys")
    raise

🔑 Validating Groq API key...
✅ API key is valid and working!
   Model: llama-3.1-8b-instant
   Response: It's nice to meet


## 2. Download SEC 10-K PDFs

Download Apple 2024 and Tesla 2023 10-K filings.

In [9]:
import os
os.makedirs("data", exist_ok=True)

# Download Apple 10-K 2024
!wget -q -O data/apple_10k_2024.pdf "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm" 2>/dev/null || echo "Using local copy"

# Download Tesla 10-K 2023
!wget -q -O data/tesla_10k_2023.pdf "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm" 2>/dev/null || echo "Using local copy"

# Check if files exist
!ls -la data/

Using local copy
Using local copy
total 3816
drwxr-xr-x@  4 indhra  staff     128 Jan 30 20:07 .
drwxr-xr-x@ 16 indhra  staff     512 Jan 30 20:07 ..
-rw-r--r--@  1 indhra  staff  963934 Jan 30 20:07 10-Q4-2024-As-Filed.pdf
-rw-r--r--@  1 indhra  staff  984581 Jan 30 20:07 tsla-20231231-gen.pdf


## 3. Initialize RAG Pipeline

Load the embedding model, reranker, and LLM.

In [10]:
from src.pipeline import RAGPipeline

# Initialize with Groq (free tier)
pipeline = RAGPipeline(
    embedding_model="bge-small",    # Fast, good quality
    reranker_model="ms-marco-mini", # Fast cross-encoder
    llm_provider="groq",            # Free API
    use_hybrid_search=True,         # Vector + BM25
    top_k_retrieval=15,             # Initial candidates (increased for better coverage)
    top_k_rerank=7                  # After reranking (increased for complex questions)
)

Initializing SEC 10-K RAG Pipeline

[1/4] Loading embedding model...
Using embedding model: bge-small
  Model: BAAI/bge-small-en-v1.5
  Dimensions: 384
  Quality: good, Speed: fast
  Device: mps


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ Model loaded successfully

[2/4] Loading reranker...
Using reranker: ms-marco-mini
  Model: cross-encoder/ms-marco-MiniLM-L-6-v2
  Quality: good, Speed: fast
  Device: mps


Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ Reranker loaded

[3/4] Initializing LLM...
GroqLLM initialized with model: llama-3.1-8b-instant

[4/4] Pipeline ready!
  Hybrid search: True
  Retrieval top-k: 15
  Rerank top-k: 7


## 4. Index Documents

Parse PDFs, chunk, embed, and build the vector index.

In [11]:
# This takes ~2 minutes on Colab (mostly embedding generation)
num_chunks = pipeline.index_documents(data_dir="data/")
print(f"\n✅ Indexed {num_chunks} chunks!")


Indexing SEC 10-K Documents

[Step 1] Parsing PDFs...
Parsing 'Apple 10-K' (121 pages)...
✓ Parsed 121 pages from Apple 10-K
Parsing 'Tesla 10-K' (130 pages)...
✓ Parsed 130 pages from Tesla 10-K

[Step 2] Chunking documents...
Chunking Apple 10-K...
  → Created 239 chunks
Chunking Tesla 10-K...
  → Created 252 chunks

✓ Total chunks: 491
  Average tokens per chunk: 376
  Min/Max tokens: 11/1050

[Step 3] Generating embeddings...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]


[Step 4] Building vector index...
VectorStore initialized
  Dimension: 384
  Hybrid search: True
  Hybrid alpha: 0.7 (vector weight)
Adding 491 chunks to vector store...
Building BM25 index...
  BM25 index built with 491 documents
✓ Vector store now contains 491 vectors

[Step 5] Saving index to outputs/index...
Saving vector store to outputs/index...
✓ Vector store saved (491 chunks)

✓ Indexing complete! 491 chunks indexed

✅ Indexed 491 chunks!


## 5. Ask Questions!

Now we can ask questions about the 10-K filings.

In [12]:
def ask(question: str):
    """Helper function to ask a question and display the answer."""
    print(f"❓ {question}")
    print("-" * 60)

    result = pipeline.answer_question(question)

    print(f"💡 {result['answer']}")
    print(f"\n📚 Sources: {result['sources']}")
    print()

In [13]:
# Test: Apple revenue
ask("What was Apple's total revenue for fiscal year 2024?")

❓ What was Apple's total revenue for fiscal year 2024?
------------------------------------------------------------
💡 ## Understand
The question is asking for Apple's total revenue for fiscal year 2024.

## Locate
The relevant information is found in context id="6" source="

📚 Sources: ['Apple 10-K', 'Item 8', 'p. 32']



In [14]:
# Test: Tesla net income
ask("What was Tesla's net income for fiscal year 2023?")

❓ What was Tesla's net income for fiscal year 2023?
------------------------------------------------------------
💡 ## Understand
The question is asking for Tesla's net income for fiscal year 2023.

## Locate
The relevant information is found in context id="4"

📚 Sources: ['Tesla 10-K', 'Item 8', 'p. 51', 'Tesla 10-K', 'Item 8', 'p. 51']



In [15]:
# Test: Apple R&D
ask("How much did Apple spend on research and development in 2024?")

❓ How much did Apple spend on research and development in 2024?
------------------------------------------------------------
💡 ## Understand
The question is asking for the amount Apple spent on research and development in 2024.

## Locate
The relevant information is found in context chunk "1" and context chunk "2".

## Extract
From context chunk "1", we have:
"Research and development
$
31,370
5 % $
29,915
14 % $
26,251
Percentage of total net sales
8%
8%
7%"

From context chunk "2", we have:
"Research and development
$
31,370
"

## Calculate
The amount spent on research and development in 2024 is already provided in the context.

## Synthesize
Apple spent $31,370 million on research and development in 2024.

## Cite

📚 Sources: ['Apple 10-K', 'Item 7', 'p. 27', 'Apple 10-K', 'Item 8', 'p. 32']



In [16]:
# Test: Tesla cash
ask("How much cash and cash equivalents did Tesla report?")

❓ How much cash and cash equivalents did Tesla report?
------------------------------------------------------------
💡 ## Understand
The question is asking for the amount of cash and cash equivalents reported by Tesla.

## Locate
The relevant context chunks are:
- <context id="1" source="[Tesla 10-K, Item 8, p. 50]"> (Consolidated Balance Sheets)
- <context id="2" source="[Tesla 10-K, Note 2, p. 65]"> (Cash and cash equivalents and restricted cash)
- <context id="5" source="[Tesla 10-K, Note 2, p. 65]"> (Cash and cash equivalents and restricted cash)

## Extract
From <context id="1" source="[Tesla 10-K, Item 8, p. 50]">, we have:
Cash and cash equivalents = $16,398 million (as of December 31, 2023)

From <context id="2" source="[Tesla 10-K, Note 2, p. 65]">, we have:
Cash and cash equivalents = $16,398 million (as of December 31, 2023)
Restricted cash included in prepaid expenses and other current assets = Not specified
Restricted cash included in other non-current assets = Not specifie

In [17]:
# Test: Out-of-scope (future prediction) - should refuse
ask("What will Apple's revenue be in 2025?")

❓ What will Apple's revenue be in 2025?
------------------------------------------------------------
💡 This question cannot be answered based on the provided documents.

📚 Sources: []



In [18]:
# Test: Out-of-scope (investment advice) - should refuse
ask("Should I invest in Tesla stock?")

❓ Should I invest in Tesla stock?
------------------------------------------------------------
💡 **Understand**: The user is asking for investment advice on Tesla stock.
**Locate**: The context does not contain explicit investment advice or recommendations.
**Extract**: The context mentions potential risks and challenges facing Tesla, such as:
* Litigation and regulatory issues (contexts 5, 6, and 7)
* Competition for key employees (context 4)
* Potential impact of government incentives and tax credits (context 3)
**Synthesize**: Based on the provided context, there is no explicit recommendation to invest in Tesla stock. The context highlights potential risks and challenges that may affect the company's performance.
**Cite**: ["Tesla 10-K", "Note 15", "p. 90"], ["Tesla 10-K", "Item 1A", "p. 21"], ["Tesla 10-K", "Item 1A", "p. 26"]

**Answer**: I cannot provide investment advice, stock recommendations, or trading guidance. The context does not contain explicit information about Tesla's 

## 6. Run Full Evaluation

Test all 13 questions from the assignment.

In [19]:
# Run the full evaluation script

!python -m src.test.evaluate

SEC 10-K RAG EVALUATION

Initializing pipeline...
Initializing SEC 10-K RAG Pipeline

[1/4] Loading embedding model...
Using embedding model: bge-small
  Model: BAAI/bge-small-en-v1.5
  Dimensions: 384
  Quality: good, Speed: fast
  Device: mps
Loading weights: 100%|█| 199/199 [00:00<00:00, 4217.79it/s, Materializing param=
BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
✓ Model loaded successfully

[2/4] Loading reranker...
Using reranker: ms-marco-mini
  Model: cross-encoder/ms-marco-MiniLM-L-6-v2
  Quality: good, Speed: fast
  Device: mps
Loading weights: 100%|█| 105/105 [00:00<00:00, 2331.37it/s, Materializing param=
BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | St

## 7. Interactive Demo

Ask your own questions!

In [20]:
# Interactive loop
while True:
    question = input("\n🔍 Enter your question (or 'quit' to exit): ")
    if question.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    ask(question)

❓ 
------------------------------------------------------------
💡 Since there is no user question provided, I will demonstrate the response structure with a sample question.

**Sample Question:** What are the production locations and statuses of Tesla's announced vehicle models in production and under development, as of the date of this Annual Report on Form 10-K?

**Understand:** The question is asking for the production locations and statuses of Tesla's vehicle models.

**Locate:** The relevant information is found in context chunk 6: ["Tesla 10-K", "Item 1", "p. 35"]

**Extract:** The production locations and statuses of Tesla's announced vehicle models are listed as follows:

1. Fremont Factory
	* Model S / Model X: Active
	* Model 3 / Model Y: Active
2. Gigafactory Shanghai
	* Model 3 / Model Y: Active
3. Gigafactory Berlin-Brandenburg
	* Model Y: Active
4. Gigafactory Texas
	* Model Y: Active

**Synthesize:** The production locations and statuses of Tesla's announced vehicle mode

KeyboardInterrupt: 

---

## System Architecture

```
┌─────────────┐    ┌──────────┐    ┌─────────┐    ┌────────────┐
│  PDF Parse  │ →  │  Chunk   │ →  │  Embed  │ →  │ FAISS+BM25 │
│  (PyMuPDF)  │    │ (512 tok)│    │  (BGE)  │    │  (Hybrid)  │
└─────────────┘    └──────────┘    └─────────┘    └────────────┘
                                                        │
                                                        ▼
┌─────────────┐    ┌──────────┐    ┌─────────┐    ┌────────────┐
│   Answer    │ ←  │   LLM    │ ←  │ Rerank  │ ←  │   Search   │
│ + Citations │    │  (Groq)  │    │(MS-MARCO│    │  Results   │
└─────────────┘    └──────────┘    └─────────┘    └────────────┘
```

## Evaluation Results

| Question | Status |
|----------|--------|
| Q1: Apple revenue | ✅ $391,035M |
| Q2: Apple shares | ✅ 15,115,823,000 |
| Q3: Apple term debt | ✅ $96,662M |
| Q4: Apple 10-K date | ✅ November 1, 2024 |
| Q5: SEC comments | ✅ Item 1B: None |
| Q6: Tesla revenue | ✅ $96,773M |
| Q7: Automotive % | ✅ 83.04% |
| Q8: Elon dependency | ⚠️ Valid interpretation |
| Q9: Tesla vehicles | ✅ Model S, 3, X, Y |
| Q10: Lease pass-through | ✅ Finance solar |
| Q11-13: Out-of-scope | ✅ Correctly refused |

**Overall: 12/13 correct (92.3%)**

See `design_report.md` for detailed explanations of each design decision.